In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero
dir_ = '../../../data/'
group_dir_ = os.path.join(dir_, 'groups/low')

In [2]:
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'
df = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

num_user = len(svd['uid'].unique())
num_user

220

In [3]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [4]:
pop_dict = {}
for i in df['tid'].unique():
    pop_dict[i] = len(df[df['tid']==i])/num_user

In [5]:
svd['count'] = 1
svd[:5]

,uid,tid,rating,count
0,0,3,2.544071,1
1,0,7,2.545822,1
2,0,18,2.152490,1
3,0,20,1.882396,1
4,0,31,2.410806,1


In [6]:
# Count POP to DF
tid_list = []
pop_list = []
for i in df['tid'].unique():
    tid_list.append(i)
    pop_list.append(len(df[df['tid']==i])/num_user*5)
    
d = {'tid': tid_list, 'rating': pop_list}
df_pop = pd.DataFrame(data=d)
df_pop = df_pop.sort_values(by=['rating'], ascending=False)
df_pop[:10]

,tid,rating
166,166,3.840909
7209,7209,3.295455
350,350,3.113636
3761,3761,3.045455
80,80,3.022727
1017,1017,2.840909
1753,1753,2.750000
428,428,2.727273
200,200,2.681818
3159,3159,2.659091


In [7]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [10]:
for lambda_ in tqdm(lambdas):
    top_n_size = 20
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = df_pop.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
    threshold = 0.0
    avg_pops = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        pops = []
        for j in (range(len(groups))):
            group = groups[j]
            top_n_list = top_n_lists[j]
            for k in range(len(group)):
                for l in range(top_n_size):
                    tid = top_n_list[l]
                    pops.append(pop_dict[tid])                    
        avg_pop = sum(pops)/len(pops)
        avg_pops.append(avg_pop)
        
    print(avg_pops)

[0.5554545454545315, 0.5554545454545387, 0.5554545454545414, 0.5554545454545425, 0.5554545454545431, 0.5554545454545438, 0.5554545454545438]
[0.5528090909090791, 0.5538567493112895, 0.5541507177033462, 0.5545629370629347, 0.5549090909090891, 0.5551988636363622, 0.5549350649350637]
[0.5455151515151443, 0.5496005509641846, 0.550753588516745, 0.5515734265734259, 0.5529090909090901, 0.5534659090909088, 0.5535389610389606]
[0.5334454545454517, 0.5426584022038566, 0.5451076555023933, 0.5469405594405602, 0.5493181818181831, 0.5502272727272735, 0.5512662337662344]
[0.5183636363636366, 0.5304683195592307, 0.5360287081339725, 0.5378846153846173, 0.5437500000000018, 0.5462215909090922, 0.5466558441558451]
[0.49983939393939736, 0.5146625344352649, 0.519569377990434, 0.5261713286713312, 0.5319772727272742, 0.5362500000000018, 0.5396428571428576]
[0.48860000000000375, 0.49480027548209615, 0.4983014354067015, 0.513444055944058, 0.5198636363636371, 0.5166477272727283, 0.5232142857142874]
[0.4751181818